## Demo queries

This notebook has example queries for RNA-seq and ATAC-seq data. It incorperates integration with meta data (currently limited to GO terms but will expand soon).

Dependencies:
* psycopg2
* pgspecial
* ipython-sql
* bokeh

You will need credentials.py in the same directory as the ipynb file to load the credentials for logging in.


In [ ]:
from credentials import username, password, hostname, dbname
connection_info = f"postgresql://{username}:{password}@{hostname}/{dbname}"
%load_ext sql
%sql $connection_info

In [ ]:
import pandas as pd
from bokeh.io import output_notebook
from bokeh.plotting import figure, output_file, show, ColumnDataSource,save
from bokeh.models import HoverTool,LabelSet, Label,CDSView, BooleanFilter 
from bokeh.palettes import Category10
from bokeh.layouts import column, row
output_notebook()

### Query 1, text search for gene name and plot timecourse

Given a text-search for a gene name (can find multiple genes), give the average expression from replicate groups in the **Israel** et al. dataset. Download the data as a csv file and make interactive plots of the average gene experession aong replicates among the three species per gene found in the text search

In [ ]:
# SQL parameters
gene_name_search = 'Myc'

# Save and plot parameters
save_location = '/Users/ZMielko/Desktop/EGAD_Test'

### SQL Code

In [ ]:
%%sql results << 

SELECT rm.species,rm.dev_stage, gi.name, ge.gene_id, AVG(ge.expression) as avg_gene_exp
FROM rna_meta rm
    INNER JOIN gene_expression ge ON ( rm.rna_id = ge.rna_id )
        INNER JOIN gene_info gi ON ( ge.gene_id = gi.gene_id )
WHERE  gi.name ~ :gene_name_search AND
    rm.analyst = 'Israel et al.'
GROUP BY rm.species,rm.dev_stage, gi.name,ge.gene_id

### Save data and make interactive plots

In [ ]:
### Run to save data and make interactive plots ###

# Save data
resultsdf = results.DataFrame()
resultsdf.to_csv(f"{save_location}/{gene_name_search}.csv", index = False)


### Make an interactive plot ###
# Prepare data for interactive plot
x_axis_conv = {"egg":1,"4cell":2,"16cell":3,"32cell":4,"Blastula":5,"Gastrula":6,"Early_Larva":7}
resultsdf['dev_stage'] = resultsdf['dev_stage'].apply(lambda x: x_axis_conv[x])
unique = resultsdf[['gene_id', 'name']].groupby(['gene_id', 'name']).size()
unique = pd.DataFrame(unique).reset_index()
gene_list = unique['gene_id']
name_list = unique['name']
by_gene = []
for gene in gene_list:
    each_gene = resultsdf.query("gene_id == @gene")
    each_gene = each_gene.sort_values(by=['species','dev_stage'])
    each_gene = each_gene.reset_index(drop=True)
    by_gene.append(each_gene)


# Plot interactive options
TOOLS= "crosshair,pan,wheel_zoom,zoom_in,zoom_out,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select,"
hover = HoverTool(tooltips=[
    ("Gene Name", "@name"),
    ("Gene ID", "@gene_id"),
    ("Average Expression", "@avg_gene_exp")
])

graphs = []
for idx, gene in enumerate(by_gene):
    # Get species and set plot source
    species_list = list(set(gene['species']))
    source = ColumnDataSource(data = gene)
    views = []
    for i in species_list:
        views.append(CDSView(source=source, filters=[BooleanFilter([True if y == i else False for y in gene['species']])]))
    # axis and dimensions information
    Title = f"{name_list[idx]}, {gene_list[idx]}"
    p = figure(plot_height=500, plot_width=800, tools=[TOOLS,hover],x_range=(0, 8), title = Title)
    p.xaxis.axis_label = "Developmental Stage"
    p.xaxis.ticker = [1,2,3,4,5,6,7]
    p.xaxis.major_label_overrides = {1:"egg",2:"4cell",3:"16cell",4:"32cell",5:"Blastula",6:"Gastrula",7:"Early_Larva"}
    # Draw lines
    colors = Category10[len(views)]
    for idx, view in enumerate(views):
        p.line(x = 'dev_stage', y= 'avg_gene_exp', color = colors[idx], source=source, view=view,
              legend = species_list[idx])
    graphs.append(p)

show(column(graphs))

### Query 2, GO term search for gene expression data

Given a text-search for a GO term (can find multiple genes), give the average expression from replicate groups in **every** stored dataset. Download the data as a csv file and make interactive plots of the expression among the three species per gene found in the text search

In [ ]:
# Parameters

GO_search = 'calcium'
save_location = '/Users/ZMielko/Desktop/EGAD_Test'

# Cannot save as Wide since the format of time vs developmental stage is different.
# Would have to seperate by how development is delineated and save into seperate files

### SQL Code

In [ ]:
%%sql results << 

SELECT rm.analyst,rm.species,rm.dev_stage, gi.name, ge.gene_id, AVG(ge.expression) as avg_gene_exp
FROM rna_meta rm
    INNER JOIN gene_expression ge ON ( rm.rna_id = ge.rna_id )
        INNER JOIN gene_info gi ON ( ge.gene_id = gi.gene_id )
            INNER JOIN gene_ontology go ON (gi.gene_id = go.gene_id )
WHERE  go.ot_data ~ :GO_search
GROUP BY rm.analyst,rm.species,rm.dev_stage, gi.name,ge.gene_id

### Save data 

In [ ]:
### Saving actual results ###
resultsdf = results.DataFrame()
resultsdf.to_csv(f"{save_location}/{GO_search}.csv", index = False)
resultsdf


### Query 3, RNA-seq and Atac-seq data

Find out which genes are within X kb of atac-seq peaks in the blastula stage of S. purpuratus. Return the gene expression data from the Israel et al. dataset for those genes


In [ ]:
# Parameters
peak_distance = 5000
save_location = '/Users/ZMielko/Desktop/EGAD_Test'

### SQL Code

In [ ]:
%%sql results <<

/* Modify the SELECT and GROUP BY statement to return specific values */
/* Ex, if you just want a list of genes, only have gi.gene_id in both */

SELECT rm.species,rm.dev_stage, gi.gene_id, gi.name, ge.expression
FROM rna_meta rm
INNER JOIN gene_expression ge ON (rm.rna_id = ge.rna_id)
    INNER JOIN gene_info gi ON (ge.gene_id = gi.gene_id)  
        INNER JOIN gene_structure gs ON ( gi.gene_id = gs.gene_id  )  
            INNER JOIN genome_structure gs1 ON ( gs.genome_structure_id = gs1.genome_structure_id  )  
                INNER JOIN atac_data ad ON ( gs1.genome_structure_id = ad.genome_structure_id  )  
                    INNER JOIN atac_meta am ON ( ad.atac_id = am.atac_id  ) 
WHERE am.dev_stage = 'Blastula' AND
    am.description = 'Whole_embryo' AND
    rm.analyst = 'Israel et al.' AND
    CASE WHEN gs.strand = '+' THEN abs(lower(gs.gene_location) - lower(ad.peak_range)) <= :peak_distance OR  abs(lower(gs.gene_location) - upper(ad.peak_range)) <= :peak_distance 
         ELSE abs(upper(gs.gene_location) - lower(ad.peak_range)) <= :peak_distance  OR  abs(upper(gs.gene_location) - upper(ad.peak_range)) <= :peak_distance 
         END
GROUP BY rm.species,rm.dev_stage, gi.gene_id, gi.name, ge.expression

### Saving data

In [ ]:
resultsdf = results.DataFrame()
resultsdf.to_csv(f'{save_location}/Query3_{peak_distance}.csv', index = False) 
resultsdf

### Query 4, differential peaks

For a differential peak analysis, find all genes within X bases and get the GO terms for a specific category, count how may times they appear and order from most common to least common

In [ ]:
peak_distance = 5000
de_dataset = 'PMC_enriched' # other option at the moment is 'Non_PMC_enriched'

In [ ]:
%%sql results <<


SELECT go.ot_data as Terms, COUNT(*) AS Occurences
FROM gene_info gi 
        INNER JOIN gene_ontology go ON (go.gene_id = gi.gene_id)
        INNER JOIN gene_structure gs ON ( gi.gene_id = gs.gene_id  )  
            INNER JOIN genome_structure gs1 ON ( gs.genome_structure_id = gs1.genome_structure_id  )  
                INNER JOIN atac_differential_peaks adp ON ( gs1.genome_structure_id = adp.genome_structure_id  )  
                    INNER JOIN atac_diff_meta adm ON ( adp.atac_diff_id = adm.atac_diff_id  ) 
WHERE adm.description = :de_dataset AND
    CASE WHEN gs.strand = '+' THEN abs(lower(gs.gene_location) - lower(adp.peak_range)) <= :peak_distance OR  abs(lower(gs.gene_location) - upper(adp.peak_range)) <= :peak_distance 
         ELSE abs(upper(gs.gene_location) - lower(adp.peak_range)) <= :peak_distance  OR  abs(upper(gs.gene_location) - upper(adp.peak_range)) <= :peak_distance 
         END
GROUP BY go.ot_data
ORDER BY
    Occurences DESC

In [ ]:
resultsdf = results.DataFrame()
resultsdf